In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"

import numpy as np
from tensorflow.keras.optimizers import Adam , RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , concatenate , Conv2D , MaxPooling2D , Activation , UpSampling2D , BatchNormalization, Conv2DTranspose
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    
    print(images.shape)
    
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = False, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def weighted_dice_coeff(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    return score


def weighted_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = 1 - weighted_dice_coeff(y_true, y_pred, weight)
    return loss


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
                                          (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + (1 - weighted_dice_coeff(y_true, y_pred, weight))
    return loss

In [ ]:
def standard_uint(input_tensor, nb_filter):
    
    x = Conv2D(nb_filter, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


def get_nest_unet_512_layer_7(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)
        pool5 = MaxPooling2D((2, 2), strides = (2, 2))(conv5_1)      
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        # 16
        conv6_1 = standard_uint(pool5, nb_filter = 512)        
        
        up5_2 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_1)
        conv5_2 = concatenate([up5_2, conv5_1], axis = 3)
        conv5_2 = standard_uint(conv5_2, nb_filter = 256) 
        
        up4_3 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_2)
        conv4_3 = concatenate([up4_3, conv4_1, conv4_2], axis = 3)
        conv4_3 = standard_uint(conv4_3, nb_filter = 128)
        
        up3_4 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_3)
        conv3_4 = concatenate([up3_4, conv3_1, conv3_2, conv3_3], axis = 3)
        conv3_4 = standard_uint(conv3_4, nb_filter = 64)
        
        up2_5 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_4)
        conv2_5 = concatenate([up2_5, conv2_1, conv2_2, conv2_3, conv2_4], axis = 3)
        conv2_5 = standard_uint(conv2_5, nb_filter = 32)   
        
        up1_6 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_5)
        conv1_6 = concatenate([up1_6, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5], axis = 3)
        conv1_6 = standard_uint(conv1_6, nb_filter = 16)   
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        nested_output_5 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_6)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4, nested_output_5]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_5])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model
    
   

def get_nest_unet_512_layer_6(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)
        pool5 = MaxPooling2D((2, 2), strides = (2, 2))(conv5_1)      
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        # 16
        conv6_1 = standard_uint(pool5, nb_filter = 512)
        pool6 = MaxPooling2D((2, 2), strides = (2, 2))(conv6_1)        
        
        up5_2 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_1)
        conv5_2 = concatenate([up5_2, conv5_1], axis = 3)
        conv5_2 = standard_uint(conv5_2, nb_filter = 256) 
        
        up4_3 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_2)
        conv4_3 = concatenate([up4_3, conv4_1, conv4_2], axis = 3)
        conv4_3 = standard_uint(conv4_3, nb_filter = 128)
        
        up3_4 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_3)
        conv3_4 = concatenate([up3_4, conv3_1, conv3_2, conv3_3], axis = 3)
        conv3_4 = standard_uint(conv3_4, nb_filter = 64)
        
        up2_5 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_4)
        conv2_5 = concatenate([up2_5, conv2_1, conv2_2, conv2_3, conv2_4], axis = 3)
        conv2_5 = standard_uint(conv2_5, nb_filter = 32)   
        
        up1_6 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_5)
        conv1_6 = concatenate([up1_6, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5], axis = 3)
        conv1_6 = standard_uint(conv1_6, nb_filter = 16)   
        
        # Center
        conv7_1 = standard_uint(pool6, nb_filter = 1024)
        
        up6_2 = Conv2DTranspose(512, (2, 2), strides = (2, 2), padding = 'same')(conv7_1)
        conv6_2 = concatenate([up6_2, conv6_1], axis = 3)
        conv6_2 = standard_uint(conv6_2, nb_filter = 512)
        
        up5_3 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_2)
        conv5_3 = concatenate([up5_3, conv5_1, conv5_2], axis = 3)
        conv5_3 = standard_uint(conv5_3, nb_filter = 256)  
        
        up4_4 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_3)
        conv4_4 = concatenate([up4_4, conv4_1, conv4_2, conv4_3], axis = 3)
        conv4_4 = standard_uint(conv4_4, nb_filter = 128)     
        
        up3_5 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_4)
        conv3_5 = concatenate([up3_5, conv3_1, conv3_2, conv3_3, conv3_4], axis = 3)
        conv3_5 = standard_uint(conv3_5, nb_filter = 64)
        
        up2_6 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_5)
        conv2_6 = concatenate([up2_6, conv2_1, conv2_2, conv2_3, conv2_4, conv2_5], axis = 3)
        conv2_6 = standard_uint(conv2_6, nb_filter = 32)    
        
        up1_7 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_6)
        conv1_7 = concatenate([up1_7, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5, conv1_6], axis = 3)
        conv1_7 = standard_uint(conv1_7, nb_filter = 16)
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        nested_output_5 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_6)
        nested_output_6 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_7)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4, nested_output_5, nested_output_6]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_6])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model


def get_nest_unet_512_layer_5(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)    
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_4])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model
    
    
    
def get_nest_unet_512_layer_5_more_channels(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)    
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_4])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model

In [ ]:
def divide_gray_imgs(images):
    
    '''
        512 x 512に分割する
        端の50pxをのりしろとする
        空欄箇所は-1（黒）にする
        height, widthが412未満の場合は変数を-1に変更
    '''
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        #cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        #cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                #print(z*H*W+ w*H+h)
                
    return diveded_imgs

In [ ]:
def make_train_datasets(ori_images, label_images, num_list):
    
    """
        Args : 
            ori_images (numpy.ndarray) : Z, Y, X, 1
            label_images (numpy.ndarray) : Z, Y, X, 1
            num_list (list) : num
            
        Returns : 
            train_ori_images (numpy.ndarray) : Z, Y, X, 1
            train_label_images (numpy.ndarray) : Z, Y, X, 1
            
        Memo:
            num_listに入っている画像を抽出
            上下左右10%は使わない
        
        
    """
    
    train_ori_images = list()
    train_label_images = list()
    for num in num_list:
        ori_image = ori_images[num]
        label_image = label_images[num]
        
        # 切り出す範囲を指定
        x_range = [int(ori_image.shape[0] * 0.1), int(ori_image.shape[0] * 0.9)]
        y_range = [int(ori_image.shape[1] * 0.1), int(ori_image.shape[1] * 0.9)]
        
        train_ori_images.append(ori_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        train_label_images.append(label_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        
    cropped_ori_images = divide_gray_imgs(np.array(train_ori_images))
    cropped_label_images = divide_gray_imgs(np.array(train_label_images))
    
    cropped_label_images = np.where(
        cropped_label_images == -1,
        0,
        cropped_label_images
    )
        
    return cropped_ori_images, cropped_label_images

In [ ]:
# U-Netのトレーニングを実行する関数
def train_unet(X_train,Y_train, csv_path, model_path ,input_shape=(512, 512, 1), num_classes=1):
    Y_train = Y_train
    X_train = X_train
    
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
    # we create two instances with the same arguments
    data_gen_args = dict(
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
    )
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    image_datagen.fit(X_train, augment=True, seed=seed)
    mask_datagen.fit(Y_train, augment=True, seed=seed)

    image_generator = image_datagen.flow(X_train, seed=seed, batch_size=8)
    mask_generator = mask_datagen.flow(Y_train, seed=seed, batch_size=8)

    # combine generators into one which yields image and masks
    train_generator = (pair for pair in zip(image_generator, mask_generator))
    
    #model = get_unet_512(input_shape=input_shape, num_classes=num_classes)
    model = get_nest_unet_512_layer_5(input_shape=input_shape, num_classes=num_classes)

    BATCH_SIZE = 4
    # 20エポック回せば十分
    NUM_EPOCH = 400
    
    callbacks = []
    from tensorflow.keras.callbacks import CSVLogger
    callbacks.append(CSVLogger(csv_path))
    history = model.fit_generator(train_generator,steps_per_epoch=32, epochs=NUM_EPOCH, verbose=1, callbacks=callbacks)
    #history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, verbose=1, callbacks=callbacks)
    model.save_weights(model_path)

# shCtrl

## cropped_001

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins//cropped_001/original//')

In [ ]:
## 初回だけ

# save_ori_imgs

out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/original'
os.makedirs(out_dir, exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    img = ori_imgs[i][y_range[0]:y_range[1], x_range[0]:x_range[1], :]
    cv2.imwrite(
        f"{out_dir}/{str(cnt).zfill(4)}.png",
        denormalize_x(img)
    )
    cnt += 1
    
img.shape

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_001//')


In [ ]:
num_lst_001 = [110, 170, 240]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins//cropped_001/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_002//')


In [ ]:
num_lst_001 = [110, 170, 240]
num_lst_002 = [156, 248]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)



In [ ]:
cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins//cropped_001/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_002//')
label_imgs_003 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_003//')


In [ ]:
num_lst_001 = [110, 170, 240]
num_lst_002 = [156, 248]
num_lst_003 = [145, 225]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)
cropped_ori_003_imgs, cropped_label_003_imgs = make_train_datasets(ori_imgs, label_imgs_003, num_lst_003)

cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs, cropped_ori_003_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs, cropped_label_003_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_003.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

## cropped_002

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins//cropped_002/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/modified_mito_001//')


In [ ]:
num_lst_001 = [56, 129, 205]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins//cropped_002/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/modified_mito_002//')


In [ ]:
num_lst_001 = [56, 129, 205]
num_lst_002 = [35, 163, 229]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_002)


In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

## cropped_003

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations//cropped_003/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_001//')


In [ ]:
num_lst_001 = [126, 169, 275]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)


In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs])
cropped_label_imgs = np.concatenate([cropped_label_001_imgs])

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations//cropped_003/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_002//')


In [ ]:
num_lst_001 = [126, 169, 275]
num_lst_002 = [121, 333, 361]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)


In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs])
cropped_label_imgs = np.concatenate([cropped_label_001_imgs])

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations//cropped_003/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_002//')
label_imgs_003 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/manually_mito_003//')


In [ ]:
num_lst_001 = [126, 169, 275]
num_lst_002 = [121, 333, 361]
num_lst_003 = [191, 237, 350]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)
cropped_ori_003_imgs, cropped_label_003_imgs = make_train_datasets(ori_imgs, label_imgs_003, num_lst_003)


In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs, cropped_ori_002_imgs, cropped_ori_003_imgs])
cropped_label_imgs = np.concatenate([cropped_label_001_imgs, cropped_label_002_imgs, cropped_label_003_imgs])

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_003.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

## cropped_005

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations//cropped_005/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/modified_mito_001//',normalize=False)


In [ ]:
num_lst_001 = [13, 31, 125, 173, 215, 242, 274]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003//models/cropped_005/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

# shOPA1

## cropped_001

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins//cropped_001/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_001//')


In [ ]:
num_lst_001 = [123, 250, 380]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins//cropped_001/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_002//')


In [ ]:
num_lst_001 = [123, 250, 380]
num_lst_002 = [154, 189, 393]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)



In [ ]:
cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins//cropped_001/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_002//')
label_imgs_003 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_003//')


In [ ]:
num_lst_001 = [123, 250, 380]
num_lst_002 = [154, 189, 393]
num_lst_003 = [114, 230, 385]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)
cropped_ori_003_imgs, cropped_label_003_imgs = make_train_datasets(ori_imgs, label_imgs_003, num_lst_003)



In [ ]:
cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs, cropped_ori_003_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs, cropped_label_003_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_001/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_003.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

## cropped_002

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_001//')


In [ ]:
num_lst_001 = [40, 150, 236]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_002/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_002//')


In [ ]:
num_lst_001 = [40, 150, 236]
num_lst_002 = [10, 95, 241]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)


In [ ]:
cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_002/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_003

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/original//')

In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_002//')
label_imgs_003 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_003//')


In [ ]:
num_lst_001 = [40, 150, 236]
num_lst_002 = [10, 95, 241]
num_lst_003 = [15, 75, 259]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)
cropped_ori_003_imgs, cropped_label_003_imgs = make_train_datasets(ori_imgs, label_imgs_003, num_lst_003)


In [ ]:
cropped_ori_imgs = np.concatenate((cropped_ori_001_imgs, cropped_ori_002_imgs, cropped_ori_003_imgs))
cropped_label_imgs = np.concatenate((cropped_label_001_imgs, cropped_label_002_imgs, cropped_label_003_imgs))

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003//models/cropped_002/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_003.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

## cropped_003

### train_001

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations//cropped_003/original//')


In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_mito_001//')


In [ ]:
num_lst_001 = [0, 57, 156, 193, 305, 361]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)


In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs])
cropped_label_imgs = np.concatenate([cropped_label_001_imgs])

In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_001.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

### train_002

In [ ]:
ori_imgs , _ = load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations//cropped_003/original//')


In [ ]:
label_imgs_001 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_mito_001//')
label_imgs_002 , _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_mito_002//')


In [ ]:
num_lst_001 = [0, 57, 156, 193, 305, 361]
num_lst_002 = [45, 79, 105, 249, 325]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)
cropped_ori_002_imgs, cropped_label_002_imgs = make_train_datasets(ori_imgs, label_imgs_002, num_lst_002)


In [ ]:
out_dir = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/mito_active_learning"
os.makedirs(out_dir, exist_ok = True)

csv_path = f'{out_dir}/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = f'{out_dir}/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)